# Anomaly Detection Model Training

Here we show an example of training 3 different models for unsupervised anomaly detection:
* Elliptical Envelope
* Local Outlier Detection
* Isolation Forest

The models are optimized via cross-validation using custom metrics defined below

Since we transform the original data via Principle Components Analysis (PCA), the preparatory steps and the model itself can be combined into a pipeline:
1) Standartization of each feature across the training set

2) PCA transformation

3) Model

## Load all modules

In [29]:
import pandas as pd
import numpy as np
from matplotlib.gridspec import GridSpec

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit, train_test_split

import random

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.covariance import EllipticEnvelope
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

from sklearn.metrics import roc_auc_score as auc
from sklearn.pipeline import Pipeline

from load_data import process_files

## Load the data

### Load the data 

Select `from_csv = True` if the data have been preprocessed and saved to a csv file already. 

Alternatevely, select `from_csv = False` if the raw hdf5 files need to be processed. in this case, indicate path to the data directory `data_dir`.

In [35]:
from_csv = True
data_dir = None # '../data/all_hdf5/*.h5'

In [36]:
if from_csv:
    df = pd.read_csv('CSX_data.csv', index_col = 0)
else:
    df = process_files(path = data_dir )

In [37]:

df_normal = df[df.target == 'normal'][:].sample(frac = 1, random_state = 10)
df_anomaly = df[df.target == 'anomaly'][:].sample(frac = 1, random_state = 10)

print(f'Normal data size: {df_normal.shape[0]}')
print(f'Anomaly data size: {df_anomaly.shape[0]}')

Normal data size: 399
Anomaly data size: 328


### Split the data
* train: 80% of normal data 
* validation: 10% of normal data and 50% of anomalous data
* test: 10% of normal data and 50% of anomalous data

In [25]:
# splitting points for nomal data
normal_split1 = (df_normal.shape[0]//10)*8
normal_split2 = (df_normal.shape[0]//10)*1 + normal_split1

train = df_normal.iloc[:normal_split1, :]
val_normal = df_normal.iloc[normal_split1:normal_split2, :]
test_normal = df_normal.iloc[normal_split2:, :]

# splitting poin for anomaly data
anomaly_split = df_anomaly.shape[0]//2

val_anomaly = df_anomaly.iloc[:anomaly_split, :]
test_anomaly = df_anomaly.iloc[anomaly_split:, :]

val = pd.concat([val_normal, val_anomaly]).reset_index(drop=True)
test = pd.concat([test_normal, test_anomaly]).reset_index(drop=True)

print(f'\n training data shape: {train.shape} \n validation data shape:{val.shape} \n test data shape: {test.shape}')


 training data shape: (312, 95) 
 validation data shape:(203, 95) 
 test data shape: (212, 95)


### Separate data into features and targets

In [26]:
X = train.drop(columns = ['target', 'roi'])
target = train['target'].map({'anomaly': -1, 'normal': 1})

X_val = val.drop(columns = ['target', 'roi'])
target_val = val['target'].map({'anomaly': -1, 'normal': 1})

X_test = test.drop(columns = ['target', 'roi'])
target_test = test['target'].map({'anomaly': -1, 'normal': 1})

### Construct (training, validation) split to use for model optimization

In [28]:
#combine training and validation features and targets for them to have consistent indexes
X_comb = pd.concat([X, X_val]).reset_index(drop=True)
target_comb = pd.concat([target, target_val]).reset_index(drop=True)


# split the sets again, now the indexes are consistent
X_train, X_val, y_train, y_val = train_test_split(X_comb, target_comb, 
                                                  train_size=X.shape[0], random_state = None,
                                                  shuffle = False)
# define the split
split_index = [-1 if x in X_train.index else 0 for x in X_comb.index]
pds = PredefinedSplit(test_fold = split_index)

## Train Models

### Define a custom performance metrics

The metric we select here tries to simultaneously increase recalls for anomalies and normal cases and reduce false classification for both labels. The metric can be used directly in grid search optimization.

In [39]:
def combined_recalls(y, y_pred):
    anomaly_recall = recall_score(-y, -y_pred)
    normal_recall = recall_score(y, y_pred)
    anomaly_precision = precision_score(-y,-y_pred)
    normal_precision = precision_score(y, y_pred)
    
    score = anomaly_recall*normal_recall*normal_precision*anomaly_precision
    return score

from sklearn.metrics import make_scorer
score = make_scorer(combined_recalls, greater_is_better=True)

For each of the models we identify the set of parameters we want to optimize and the range of the values we would like to consider. The search for the optimal set of the parameters' values is done via GridSearchCV ( exhaustive search and cross-validation).

# Elliptic Envelope

For elliptical envelope we only vary the number of principle components and the contamination level

In [103]:
params = {'pca__n_components':np.arange(1,40),
         'model__contamination': np.linspace(0, 0.5, 20)}


pipe = Pipeline([('scaler', StandardScaler()),
                 ('pca', PCA()),
                 ('model', EllipticEnvelope(assume_centered=False, random_state = 50))]) 


### Search for optimal parameters

In [104]:
grid = GridSearchCV(pipe, params, n_jobs = -1, cv=pds, scoring = score, refit=False)
_= grid.fit(X_comb,target_comb)

### Train the model with the optimal parameters

In [106]:
contamination = grid.best_params_['model__contamination']
n_pca = grid.best_params_['pca__n_components']
best_EE = Pipeline([('scaler', StandardScaler()),
                 ('pca', PCA(n_pca)),
                 ('model', EllipticEnvelope(assume_centered=False, random_state = 50, contamination =contamination))]) 
best_EE.fit(X_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=3)),
                ('model',
                 EllipticEnvelope(contamination=0.13157894736842105,
                                  random_state=50))])

### Test the model
We consider confusion matrix and precision and false discovery rate of anomalous data

In [107]:
cmat = confusion_matrix(target_test, best_EE.predict(X_test))
print(
f'            anomaly_M   normal_M\n \
anomaly_T    {cmat[0][0]}        {cmat[0][1]}\n \
normal_T       {cmat[1][0]}       {cmat[1][1]}')

            anomaly_M   normal_M
 anomaly_T    161        3
 normal_T       6       42


In [108]:
print(f'recall = {recall_score(-target_test,-best_EE.predict(X_test)):2.1%}')
print(f'false discovery rate = {1 - precision_score(-target_test,-best_EE.predict(X_test)):2.1%}')
print(f'roc_auc = {auc(-target_test,-best_EE.predict(X_test)):1.2f}')

recall = 98.2%
false discovery rate = 3.6%
roc_auc = 0.93


# Local Outlier Detector

In [109]:
params = {'pca__n_components':np.arange(1,20),
          'model__contamination': np.linspace(0.0, 0.5, 30),
          'model__n_neighbors':np.arange(1, 25, 2),
          'model__leaf_size' : np.arange(1, 40, 5)}

pipe = Pipeline([('scaler', StandardScaler()),
                 ('pca', PCA()),
                 ('model', LocalOutlierFactor(novelty = True, algorithm='kd_tree'))]) 

### Search for optimal parameters

In [94]:
grid = GridSearchCV(pipe, params, n_jobs = -1, cv=pds, scoring = score, refit=False)
_ = grid.fit(X_comb,target_comb)

### Train the model with the optimal parameters

In [99]:
params = grid.best_params_
best_LOD = Pipeline([('scaler', StandardScaler()),
                 ('pca', PCA(params['pca__n_components'])),
                 ('model', LocalOutlierFactor(novelty = True, algorithm ='kd_tree',
                                             contamination = params['model__contamination'],
                                             n_neighbors = params['model__n_neighbors'],
                                             leaf_size = params['model__leaf_size']))]) 
best_LOD.fit(X_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=3)),
                ('model',
                 LocalOutlierFactor(algorithm='kd_tree',
                                    contamination=0.10344827586206896,
                                    leaf_size=1, n_neighbors=7,
                                    novelty=True))])

### Test the model

In [100]:
cmat = confusion_matrix(target_test, best_LOD.predict(X_test))
print(
f'            anomaly_M   normal_M\n \
anomaly_T    {cmat[0][0]}        {cmat[0][1]}\n \
normal_T       {cmat[1][0]}       {cmat[1][1]}')

            anomaly_M   normal_M
 anomaly_T    152        12
 normal_T       5       43


In [101]:
print(f'recall = {recall_score(-target_test,-best_LOD.predict(X_test)):2.1%}')
print(f'false discovery rate = {1 - precision_score(-target_test,-best_LOD.predict(X_test)):2.1%}')
print(f'roc_auc = {auc(-target_test,-best_LOD.predict(X_test)):1.2f}')

recall = 92.7%
false discovery rate = 3.2%
roc_auc = 0.91


# Isolation Forest

Unlike the above models, the number of avaiable model parameters (particularly, the maximum number of features for building an isolation tree) for Isolation Forest is dependent on the number of principle components. Thus, we change the number of components in a `for`-loop and run a grid search at each step of the loop and keep track of the best performing model.

In [123]:
best_pca = 0
best_score = 0

for j in range(1, 5):
      
    params = {'model__max_features': np.arange(1, j+1),
              'model__contamination': np.linspace(0, 0.5, 20)}
    
    
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('pca', PCA(j)),
                 ('model', IsolationForest(
                      
                      n_estimators = 300,                      
                      bootstrap=True,
                      n_jobs=-1,
                      behaviour='deprecated',
                      random_state=10,
                      verbose=0,
                      warm_start=False))]) 
    
    grid = GridSearchCV(pipe, params, n_jobs = -1, cv=pds, scoring = score, refit = False)
    grid.fit(X_comb,target_comb)
    params = grid.best_params_
    
    pipe = Pipeline([('scaler', StandardScaler()),
                 ('pca', PCA(j)),
                 ('model', IsolationForest(
                      max_samples='auto',
                      n_estimators = 300,                      
                      bootstrap=True,
                      n_jobs=-1,
                      behaviour='deprecated',
                      random_state=10,
                      verbose=0,
                      warm_start=True,
                 max_features = params['model__max_features'],
                 contamination = params['model__contamination']))]).fit(X_train)
    

    
    if grid.best_score_ > best_score:
        best_score = grid.best_score_
        best_pca = j
        best_pipe = pipe


In [124]:
cmat = confusion_matrix(target_test, best_IFT.predict(X_test))
print(
f'            anomaly_M   normal_M\n \
anomaly_T    {cmat[0][0]}        {cmat[0][1]}\n \
normal_T       {cmat[1][0]}       {cmat[1][1]}')

            anomaly_M   normal_M
 anomaly_T    162        2
 normal_T       11       37


In [125]:
print(f'recall = {recall_score(-target_test,-best_IFT.predict(X_test)):2.1%}')
print(f'false discovery rate = {1 - precision_score(-target_test,-best_IFT.predict(X_test)):2.1%}')
print(f'roc_auc = {auc(-target_test,-best_IFT.predict(X_test)):1.2f}')

recall = 98.8%
false discovery rate = 6.4%
roc_auc = 0.88


# Testing model on new data point

Select a model among the best predictors based on your preference

In [129]:
model = best_EE
new_data_file = '../data/all_hdf5/EELKGWIGY.h5' # this file was used for the model, here is considered for demostration only

new_data = process_files(path = new_data_file).drop(columns = ['target', 'roi']).values
model.predict(new_data)

array([-1])